In [1]:
from aframe import AFrame
import pandas as pd
from IPython.display import display, HTML
from pandas.io.json import json_normalize
import json
from pandas.io.json import json_normalize
import findspark
findspark.init()
import pyspark
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
import pyspark.sql.functions as func
from pyspark.sql.window import Window

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px solid black;
}

# Pandas

In [3]:
cols = ['id', 'ename', 'job', 'mgr_id', 'hiredate', 'sal', 'comm', 'deptno']
data = [(1, "SMITH", "CLERK", 8, "17-Dec-80", 800, 20, 10),
(2, "ALLEN", "SALESMAN", 6, "20-Feb-81", 1600, 300, 30),
(3, "WARD", "SALESMAN", 6, "22-Feb-81", 1250, 500, 30),
(4, "JONES", "MANAGER", 9, "2-Apr-81", 2975, 0, 20),
(5, "MARTIN", "SALESMAN", 6, "28-Sep-81", 1250, 1400, 30),
(6, "BLAKE", "MANAGER", 9, "1-May-81", 2850, 0, 30),
(7, "CLARK", "MANAGER", 9, "9-Jun-81", 2450, 0, 10),
(8, "SCOTT", "ANALYST", 4, "19-Apr-87", 3000, 0, 20),
(9, "KING", "PRESIDENT", 0, "17-Nov-81", 5000, 0, 10),
(10, "TURNER", "SALESMAN", 6, "8-Sep-81", 1500, 0, 30),
(11, "ADAMS", "CLERK", 8, "23-May-87", 1100, 0, 20)]

In [4]:
pd_df = pd.DataFrame(data, columns= cols)

In [5]:
pd_df

,id,ename,job,mgr_id,hiredate,sal,comm,deptno
0,1,SMITH,CLERK,8,17-Dec-80,800,20,10
1,2,ALLEN,SALESMAN,6,20-Feb-81,1600,300,30
2,3,WARD,SALESMAN,6,22-Feb-81,1250,500,30
3,4,JONES,MANAGER,9,2-Apr-81,2975,0,20
4,5,MARTIN,SALESMAN,6,28-Sep-81,1250,1400,30
5,6,BLAKE,MANAGER,9,1-May-81,2850,0,30
6,7,CLARK,MANAGER,9,9-Jun-81,2450,0,10
7,8,SCOTT,ANALYST,4,19-Apr-87,3000,0,20
8,9,KING,PRESIDENT,0,17-Nov-81,5000,0,10
9,10,TURNER,SALESMAN,6,8-Sep-81,1500,0,30


In [6]:
tmp = pd_df[['id', 'mgr_id', 'sal']]
tmp

,id,mgr_id,sal
0,1,8,800
1,2,6,1600
2,3,6,1250
3,4,9,2975
4,5,6,1250
5,6,9,2850
6,7,9,2450
7,8,4,3000
8,9,0,5000
9,10,6,1500


In [7]:
tmp.rolling(2, on='mgr_id').sum()

,id,mgr_id,sal
0,NaN,8,NaN
1,3.0,6,2400.0
2,5.0,6,2850.0
3,7.0,9,4225.0
4,9.0,6,4225.0
5,11.0,9,4100.0
6,13.0,9,5300.0
7,15.0,4,5450.0
8,17.0,0,8000.0
9,19.0,6,6500.0


In [8]:
pd_df['sum'] = pd_df['sal'].rolling(window=2).sum()
pd_df

,id,ename,job,mgr_id,hiredate,sal,comm,deptno,sum
0,1,SMITH,CLERK,8,17-Dec-80,800,20,10,NaN
1,2,ALLEN,SALESMAN,6,20-Feb-81,1600,300,30,2400.0
2,3,WARD,SALESMAN,6,22-Feb-81,1250,500,30,2850.0
3,4,JONES,MANAGER,9,2-Apr-81,2975,0,20,4225.0
4,5,MARTIN,SALESMAN,6,28-Sep-81,1250,1400,30,4225.0
5,6,BLAKE,MANAGER,9,1-May-81,2850,0,30,4100.0
6,7,CLARK,MANAGER,9,9-Jun-81,2450,0,10,5300.0
7,8,SCOTT,ANALYST,4,19-Apr-87,3000,0,20,5450.0
8,9,KING,PRESIDENT,0,17-Nov-81,5000,0,10,8000.0
9,10,TURNER,SALESMAN,6,8-Sep-81,1500,0,30,6500.0


# Spark

In [9]:
sc = SparkContext(appName="app")
sqlContext = SQLContext(sc)
employee = sqlContext.read.json('/Users/gift/github/Data/employee.json')

In [10]:
employee.columns

['comm', 'deptno', 'ename', 'hiredate', 'id', 'job', 'mgr_id', 'sal']

In [11]:
employee.toPandas().head(2)

,comm,deptno,ename,hiredate,id,job,mgr_id,sal
0,20,10,SMITH,17-Dec-80,1,CLERK,8.0,800
1,300,30,ALLEN,20-Feb-81,2,SALESMAN,6.0,1600


In [12]:
window = Window.partitionBy('mgr_id').orderBy(employee['sal'].desc())

In [13]:
employee.select(func.sum(employee['sal']).over(window)).toPandas()

,sum(sal) OVER (PARTITION BY mgr_id ORDER BY sal DESC NULLS LAST unspecifiedframe$())
0,1100
1,1900
2,5000
3,3000
4,1600
5,3100
6,5600
7,5600
8,2975
9,5825


In [14]:
window = Window.orderBy(employee['sal'].desc()).rowsBetween(-1,0)
employee.withColumn('sum_sal', func.sum(employee['sal']).over(window)).toPandas()

,comm,deptno,ename,hiredate,id,job,mgr_id,sal,sum_sal
0,0,10,KING,17-Nov-81,9,PRESIDENT,NaN,5000,5000
1,0,20,SCOTT,19-Apr-87,8,ANALYST,4.0,3000,8000
2,0,20,JONES,2-Apr-81,4,MANAGER,9.0,2975,5975
3,0,30,BLAKE,1-May-81,6,MANAGER,9.0,2850,5825
4,0,10,CLARK,9-Jun-81,7,MANAGER,9.0,2450,5300
5,300,30,ALLEN,20-Feb-81,2,SALESMAN,6.0,1600,4050
6,0,30,TURNER,8-Sep-81,10,SALESMAN,6.0,1500,3100
7,500,30,WARD,22-Feb-81,3,SALESMAN,6.0,1250,2750
8,1400,30,MARTIN,28-Sep-81,5,SALESMAN,6.0,1250,2500
9,0,20,ADAMS,23-May-87,11,CLERK,8.0,1100,2350


In [15]:
func.max(employee['sal']).over(window) - employee['sal']

Column<b'(max(sal) OVER (ORDER BY sal DESC NULLS LAST ROWS BETWEEN -1 FOLLOWING AND CURRENT ROW) - sal)'>

# AFrame

In [16]:
from aframe import window as wd

In [17]:
af = AFrame(dataverse='testData', dataset='employee')

In [18]:
ordered_af = af.rolling(on='sal').sum()

In [19]:
af['sum_sal'] = ordered_af
af.toPandas()

,comm,deptno,ename,hiredate,id,job,mgr_id,sal,sum_sal
0,20,10,SMITH,17-Dec-80,1,CLERK,8.0,800,800
1,0,20,ADAMS,23-May-87,11,CLERK,8.0,1100,1900
2,500,30,WARD,22-Feb-81,3,SALESMAN,6.0,1250,4400
3,1400,30,MARTIN,28-Sep-81,5,SALESMAN,6.0,1250,4400
4,0,30,TURNER,8-Sep-81,10,SALESMAN,6.0,1500,5900
5,300,30,ALLEN,20-Feb-81,2,SALESMAN,6.0,1600,7500
6,0,10,CLARK,9-Jun-81,7,MANAGER,9.0,2450,9950
7,0,30,BLAKE,1-May-81,6,MANAGER,9.0,2850,12800
8,0,20,JONES,2-Apr-81,4,MANAGER,9.0,2975,15775
9,0,20,SCOTT,19-Apr-87,8,ANALYST,4.0,3000,18775


In [20]:
af2 = AFrame(dataverse='testData', dataset='employee')

In [21]:
window = wd.Window().partitionBy('job').orderBy('sal', order='DESC')

In [22]:
ordered_af2 = af2.rolling(window=window).row_number()

In [23]:
ordered_af2.query

'SELECT VALUE ROW_NUMBER() OVER(PARTITION BY job ORDER BY sal DESC ) FROM testData.employee t;'

In [24]:
ordered_af2.collect()

,0
0,1
1,1
2,2
3,1
4,2
5,3
6,1
7,1
8,2
9,3


In [25]:
af2['row_number'] = ordered_af2
af2.head(10)

,comm,deptno,ename,hiredate,id,job,mgr_id,row_number,sal
0,0,20,SCOTT,19-Apr-87,8,ANALYST,4.0,1,3000
1,0,20,ADAMS,23-May-87,11,CLERK,8.0,1,1100
2,20,10,SMITH,17-Dec-80,1,CLERK,8.0,2,800
3,0,20,JONES,2-Apr-81,4,MANAGER,9.0,1,2975
4,0,30,BLAKE,1-May-81,6,MANAGER,9.0,2,2850
5,0,10,CLARK,9-Jun-81,7,MANAGER,9.0,3,2450
6,0,10,KING,17-Nov-81,9,PRESIDENT,NaN,1,5000
7,300,30,ALLEN,20-Feb-81,2,SALESMAN,6.0,1,1600
8,0,30,TURNER,8-Sep-81,10,SALESMAN,6.0,2,1500
9,500,30,WARD,22-Feb-81,3,SALESMAN,6.0,3,1250


In [26]:
af2[['id','job','row_number']].head()

,id,job,row_number
0,8,ANALYST,1
1,11,CLERK,1
2,1,CLERK,2
3,4,MANAGER,1
4,6,MANAGER,2


In [27]:
af2[['id','job','row_number']].query

'SELECT t.id, t.job, t.row_number FROM (SELECT t.*, ROW_NUMBER() OVER(PARTITION BY job ORDER BY sal DESC ) row_number FROM testData.employee t) t;'

In [28]:
af3 = AFrame(dataverse='testData', dataset='employee')

In [29]:
ordered_af3 = af3.rolling(on='job').rank()
ordered_af3.head()

,0
0,1
1,2
2,2
3,4
4,4


In [30]:
ordered_af3 = af3.rolling(on='job').dense_rank()
ordered_af3.head()

,0
0,1
1,2
2,2
3,3
4,3
